In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [5]:
DATASET_LOCATION = '/content/drive/MyDrive/ai-playground/ra-text-classification-spectrogram/spectrograms/bert-isot4000'
data_dir = pathlib.Path(DATASET_LOCATION)

In [6]:
TRAIN_FRACTION = 0.8
batch_size = 32 # default batch size of data
img_height = 224 # default height to which image is resized
img_width = 224 # default width to which image is resized
seed = 2022 # random seed for shuffling and transformations

In [7]:
ds_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "training",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 2983 files belonging to 2 classes.
Using 2387 files for training.


In [8]:
ds_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "validation",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 2983 files belonging to 2 classes.
Using 596 files for validation.


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = ds_val.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
normalization_layer = layers.Rescaling(1./255)


In [11]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.051980887 1.0


In [12]:
class_names = ds_train.class_names
num_classes = len(class_names)
print(class_names)

['Fake', 'True']


In [13]:
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'],)

In [ ]:
epochs=100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/100
75/75 [==============================] - 170s 2s/step - loss: 0.7758 - accuracy: 0.5450 - val_loss: 0.6746 - val_accuracy: 0.4664
Epoch 2/100
75/75 [==============================] - 124s 2s/step - loss: 0.4734 - accuracy: 0.7796 - val_loss: 0.3317 - val_accuracy: 0.8540
Epoch 3/100
75/75 [==============================] - 121s 2s/step - loss: 0.2731 - accuracy: 0.8923 - val_loss: 0.2270 - val_accuracy: 0.9128
Epoch 4/100
75/75 [==============================] - 120s 2s/step - loss: 0.1950 - accuracy: 0.9212 - val_loss: 0.2218 - val_accuracy: 0.9178
Epoch 5/100
75/75 [==============================] - 119s 2s/step - loss: 0.1761 - accuracy: 0.9305 - val_loss: 0.1811 - val_accuracy: 0.9379
Epoch 6/100
75/75 [==============================] - 121s 2s/step - loss: 0.1527 - accuracy: 0.9422 - val_loss: 0.1627 - val_accuracy: 0.9430
Epoch 7/100
75/75 [==============================] - 121s 2s/step - loss: 0.1594 - accuracy: 0.9372 - val_loss: 0.1809 - val_accuracy: 0.9346
Epoch 

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('vgg 16 model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: ignored